In [9]:
%load_ext autoreload
%autoreload 2

import os
import sys
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

sys.path.append('../')
from vocal_patterns.interface import main


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
## Get the data

download_path = "../vocal_patterns/data/dataset_tags.csv"
data = pd.read_csv(download_path)
data.head()


,path,exercise,technique,filename
0,/Users/denis/code/ElsaGregoire/vocal_patterns/...,Other,vibrato,m6_row_vibrato.wav
1,/Users/denis/code/ElsaGregoire/vocal_patterns/...,Other,vibrato,m6_caro_vibrato.wav
2,/Users/denis/code/ElsaGregoire/vocal_patterns/...,Other,vibrato,m6_dona_vibrato.wav
3,/Users/denis/code/ElsaGregoire/vocal_patterns/...,Other,straight,m6_caro_straight.wav
4,/Users/denis/code/ElsaGregoire/vocal_patterns/...,Other,straight,m6_row_straight.wav


In [4]:
## Separate the data

X = data.drop(columns=['exercise', "technique", "filename"])
y = data[['exercise']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

X_train_processed = preprocess_audio(X_train)
X_train_reshaped = X_train_processed.reshape(len(X_train_processed), 128, 259, 1)

X_test_processed = preprocess_audio(X_test)
X_test_reshaped = X_test_processed.reshape(len(X_test_processed), 128, 259, 1)

label_encoder = LabelEncoder()
y_train_labels = label_encoder.fit_transform(np.ravel(y_train))
y_train_cat = to_categorical(y_train_labels, num_classes=3)

y_test_labels = label_encoder.transform(np.ravel(y_test))
y_test_cat = to_categorical(y_test_labels, num_classes=3)


In [17]:
model = main.train()


✅ Model initialized
✅ Model compiled
Epoch 1/20
65/65 [==============================] - 3s 39ms/step - loss: 0.4351 - accuracy: 0.8279 - val_loss: 1.0336 - val_accuracy: 0.3954
Epoch 2/20
65/65 [==============================] - 2s 37ms/step - loss: 0.0500 - accuracy: 0.9822 - val_loss: 0.9491 - val_accuracy: 0.6392
Epoch 3/20
65/65 [==============================] - 2s 38ms/step - loss: 0.0205 - accuracy: 0.9928 - val_loss: 0.8969 - val_accuracy: 0.5374
Epoch 4/20
65/65 [==============================] - 2s 38ms/step - loss: 0.0055 - accuracy: 0.9990 - val_loss: 0.6729 - val_accuracy: 0.7006
Epoch 5/20
65/65 [==============================] - 2s 37ms/step - loss: 9.7845e-04 - accuracy: 1.0000 - val_loss: 0.4804 - val_accuracy: 0.7946
Epoch 6/20
65/65 [==============================] - 2s 38ms/step - loss: 5.1979e-04 - accuracy: 1.0000 - val_loss: 0.2982 - val_accuracy: 0.9040
Epoch 7/20
65/65 [==============================] - 2s 37ms/step - loss: 3.8005e-04 - accuracy: 1.0000 - val_

In [21]:
model.evaluate(X_test_reshaped, y_test_cat)


34/34 [==============================] - 0s 14ms/step - loss: 0.0507 - accuracy: 0.9862


[0.05068179965019226, 0.9861623644828796]

In [22]:
y_pred = main.predict(X_test)



Load latest model from local registry...

Load latest model from disk...
✅ Model loaded from local disk
34/34 [==============================] - 1s 14ms/step


In [23]:
y_pred


array([[3.4940960e-05, 2.8766951e-02, 9.7119808e-01],
       [2.4407591e-04, 9.9933833e-01, 4.1758514e-04],
       [9.9999869e-01, 1.3415499e-06, 3.5745536e-08],
       ...,
       [9.8739606e-01, 3.2558408e-05, 1.2571273e-02],
       [9.9887496e-01, 1.1104856e-03, 1.4563908e-05],
       [4.6700532e-09, 1.4721586e-03, 9.9852777e-01]], dtype=float32)

In [24]:
category_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

# Display the mapping
print("Category Mapping:")
for category, numerical_representation in category_mapping.items():
    print(f"{category}: {numerical_representation}")


Category Mapping:
Other: 0
arpeggios: 1
scales: 2


In [25]:
image_to_predict = np.expand_dims(X_test_reshaped[0], axis=0)
predictions = model.predict(image_to_predict)


1/1 [==============================] - 0s 13ms/step


In [26]:
predicted_indices = np.argmax(predictions, axis=1)
predicted_indices[0]


2

In [27]:
data.iloc[0,:]


path         /Users/denis/code/ElsaGregoire/vocal_patterns/...
exercise                                                 Other
technique                                              vibrato
filename                                    m6_row_vibrato.wav
Name: 0, dtype: object

In [28]:
data


,path,exercise,technique,filename
0,/Users/denis/code/ElsaGregoire/vocal_patterns/...,Other,vibrato,m6_row_vibrato.wav
1,/Users/denis/code/ElsaGregoire/vocal_patterns/...,Other,vibrato,m6_caro_vibrato.wav
2,/Users/denis/code/ElsaGregoire/vocal_patterns/...,Other,vibrato,m6_dona_vibrato.wav
3,/Users/denis/code/ElsaGregoire/vocal_patterns/...,Other,straight,m6_caro_straight.wav
4,/Users/denis/code/ElsaGregoire/vocal_patterns/...,Other,straight,m6_row_straight.wav
...,...,...,...,...
3608,/Users/denis/code/ElsaGregoire/vocal_patterns/...,arpeggios,fast_forte,m6_arpeggios_c_fast_forte_i.wav
3609,/Users/denis/code/ElsaGregoire/vocal_patterns/...,arpeggios,fast_forte,m6_arpeggios_c_fast_forte_e.wav
3610,/Users/denis/code/ElsaGregoire/vocal_patterns/...,arpeggios,fast_forte,m6_arpeggios_c_fast_forte_u.wav
3611,/Users/denis/code/ElsaGregoire/vocal_patterns/...,arpeggios,fast_forte,m6_arpeggios_f_fast_forte_u.wav
